**WordPiece**

The goal of WordPiece is to build a fixed vocabulary \(V\) of subwords such that the likelihood of the training corpus under a simple model is maximized.

Formally, if a piece of text is segmented into tokens \(t_1, t_2, \ldots, t_m\), the model assumes:

$$
P(\text{text}) \approx \prod_{i=1}^{m} P(t_i)
$$

We want to find a vocabulary \(V\) and probabilities \(P(t)\) that maximize the likelihood of the training data (or equivalently minimize the negative log-likelihood).

In practice, WordPiece is implemented as a greedy / incremental procedure where candidate merges (subwords) are added step by step, based on how much they improve the likelihood of the corpus.


Before running WordPiece:

1. Normalize text (e.g., Unicode NFKC, remove accents if desired).

2. Decide whether to lowercase (BERT-base uncased lowercased everything).

3. Perform initial whitespace tokenization (split into words).

4. Start the vocabulary with all individual characters so that no input ever becomes completely unknown.

In [2]:
from collections import Counter, defaultdict
import math
import re
import unicodedata
from typing import Dict, Iterable, List, Sequence, Set, Tuple


# -----------------------------
# Text normalization utilities
# -----------------------------

def normalize_text(text: str, lowercase: bool = True) -> str:
    """Normalize unicode (NFKC) and optionally lowercase the text."""
    text = unicodedata.normalize("NFKC", text)
    return text.lower() if lowercase else text


def whitespace_tokenize(text: str) -> List[str]:
    """Split text on whitespace into word tokens (keeps only non-empty tokens)."""
    return [t for t in re.split(r"\s+", text) if t]


# ---------------------------------------------
# WordPiece core: segmentation and training loop
# ---------------------------------------------

SPECIAL_TOKENS = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]


def split_word_into_char_pieces(word: str) -> List[str]:
    """Initial segmentation: first character is standalone, rest are continuation with '##'."""
    if not word:
        return []
    pieces: List[str] = [word[0]]
    for ch in word[1:]:
        pieces.append("##" + ch)
    return pieces


def initial_segmentation(words: Sequence[str]) -> List[List[str]]:
    """Segment every word into character-level WordPiece pieces."""
    return [split_word_into_char_pieces(w) for w in words]


def count_tokens(segmentation: Sequence[Sequence[str]]) -> Counter:
    """Count token frequencies across all segmented words."""
    counts: Counter = Counter()
    for pieces in segmentation:
        counts.update(pieces)
    return counts


def compute_log_likelihood(counts: Counter) -> float:
    """Compute log-likelihood under MLE p(t) = count(t)/sum(counts)."""
    total = sum(counts.values())
    if total == 0:
        return 0.0
    loglik = 0.0
    for tok, c in counts.items():
        p = c / total
        loglik += c * math.log(max(p, 1e-12))
    return loglik


def collect_candidate_substrings(
    words: Sequence[str],
    vocab: Set[str],
    min_freq: int,
    max_subword_len: int,
) -> Counter:
    """
    Collect candidate substrings inside words. For position i==0 we consider raw substring,
    for i>0 we consider its '##' prefixed form. We return only those not already in vocab.
    """
    candidates: Counter = Counter()
    for word in words:
        n = len(word)
        for i in range(n):
            for j in range(i + 1, min(n, i + max_subword_len) + 1):
                piece = word[i:j]
                cand = piece if i == 0 else "##" + piece
                if cand in vocab:
                    continue
                candidates[cand] += 1
    # filter by minimum frequency
    filtered = Counter({tok: c for tok, c in candidates.items() if c >= min_freq})
    return filtered


def simulate_replace_and_count(
    words: Sequence[str],
    candidate: str,
) -> Counter:
    """
    Simulate adding a candidate token by re-segmenting all words using characters + this candidate.
    We use a left-to-right greedy pass per word. Returns the resulting token counts.
    """
    counts: Counter = Counter()

    # Candidate specifics
    if candidate.startswith("##"):
        cand_raw = candidate[2:]
    else:
        cand_raw = candidate

    for word in words:
        i = 0
        n = len(word)
        first = True
        while i < n:
            # decide whether candidate can match here
            if first and not candidate.startswith("##"):
                # try to match non-prefixed candidate at start position
                if word.startswith(cand_raw, i):
                    counts[candidate] += 1
                    i += len(cand_raw)
                    first = False
                    continue
            elif (not first) and candidate.startswith("##"):
                # try to match continued candidate at non-initial position
                if word.startswith(cand_raw, i):
                    counts[candidate] += 1
                    i += len(cand_raw)
                    first = False
                    continue
            # fallback to emitting a single char piece
            ch = word[i]
            tok = ch if first else "##" + ch
            counts[tok] += 1
            i += 1
            first = False
    return counts


def build_initial_vocab(words: Sequence[str]) -> Set[str]:
    """Create initial vocabulary from characters (both standalone and '##' forms) plus special tokens."""
    chars: Set[str] = set()
    for w in words:
        chars.update(list(w))
    vocab: Set[str] = set(SPECIAL_TOKENS)
    for ch in chars:
        vocab.add(ch)
        vocab.add("##" + ch)
    return vocab


def train_wordpiece(
    corpus_texts: Iterable[str],
    vocab_target_size: int,
    min_freq: int = 2,
    max_subword_len: int = 12,
    lowercase: bool = True,
) -> Tuple[Set[str], Dict[str, float]]:
    """
    Train a simple WordPiece vocabulary using a greedy likelihood-improvement proxy.

    - Normalize and whitespace-tokenize input texts
    - Start from character-level segmentation
    - Iteratively add the candidate token that maximizes log-likelihood under MLE
    """
    # 1) Normalize and split to words
    words: List[str] = []
    for text in corpus_texts:
        text = normalize_text(text, lowercase=lowercase)
        words.extend(whitespace_tokenize(text))
    words = [w for w in words if w]

    if not words:
        # Empty corpus → return just special tokens
        vocab = set(SPECIAL_TOKENS)
        probs = {tok: 1.0 / len(vocab) for tok in vocab}
        return vocab, probs

    # 2) Initialize vocabulary with characters and specials
    vocab: Set[str] = build_initial_vocab(words)

    # 3) Initial segmentation and counts
    segmentation = initial_segmentation(words)
    counts = count_tokens(segmentation)
    current_loglik = compute_log_likelihood(counts)

    # Greedy loop
    while len(vocab) < vocab_target_size:
        candidates = collect_candidate_substrings(
            words=words, vocab=vocab, min_freq=min_freq, max_subword_len=max_subword_len
        )
        if not candidates:
            break

        best_gain = float("-inf")
        best_candidate = None

        # Evaluate each candidate by simulated re-segmentation
        for cand in candidates:
            new_counts = simulate_replace_and_count(words, cand)
            new_loglik = compute_log_likelihood(new_counts)
            gain = new_loglik - current_loglik
            if gain > best_gain:
                best_gain = gain
                best_candidate = cand

        if best_candidate is None or best_gain <= 0:
            # No candidate improves likelihood → stop early
            break

        # Commit best candidate: rebuild counts from simulation to stay consistent
        vocab.add(best_candidate)
        counts = simulate_replace_and_count(words, best_candidate)
        current_loglik = compute_log_likelihood(counts)

    # Convert counts to probabilities (MLE)
    total = sum(counts.values()) or 1
    probs: Dict[str, float] = {t: c / total for t, c in counts.items()}
    return vocab, probs


# --------------------------------
# Inference: greedy tokenization API
# --------------------------------

def tokenize_word(word: str, vocab: Set[str], unk_token: str = "[UNK]") -> List[str]:
    """
    Greedy longest-match WordPiece tokenization for a single word.
    For non-initial pieces we try '##' + piece first, then fallback to characters.
    """
    if not word:
        return []

    i = 0
    tokens: List[str] = []
    n = len(word)
    while i < n:
        end = n
        found = False
        while end > i:
            piece = word[i:end]
            if i > 0:
                piece_with_prefix = "##" + piece
                if piece_with_prefix in vocab:
                    tokens.append(piece_with_prefix)
                    i = end
                    found = True
                    break
            if piece in vocab:
                tokens.append(piece)
                i = end
                found = True
                break
            end -= 1
        if not found:
            # If nothing matches, emit [UNK] and stop for this word
            return [unk_token]
    return tokens


def tokenize(text: str, vocab: Set[str], lowercase: bool = True, unk_token: str = "[UNK]") -> List[str]:
    """Tokenize full text: normalize → split into words → apply greedy WordPiece per word."""
    text = normalize_text(text, lowercase=lowercase)
    words = whitespace_tokenize(text)
    output: List[str] = []
    for w in words:
        output.extend(tokenize_word(w, vocab, unk_token=unk_token))
    return output


# -----------------------
# Minimal usage example
# -----------------------
# Note: Uncomment to try in the notebook.
corpus = [
    "Playing players play playful plays",
    "Playground played",
]
vocab, probs = train_wordpiece(corpus, vocab_target_size=200, min_freq=2, max_subword_len=10)
print(len(vocab))
print(tokenize("players", vocab))


34
['play', '##e', '##r', '##s']


✅ Strengths

1. Greatly reduces OOV (Out-of-Vocabulary) issues

    Any word can be split into subwords or even characters, so [UNK] is rare.

    Very useful for morphologically rich languages (like Persian, German).

2. Smaller and more efficient vocabulary

    Instead of millions of words, ~30k subwords are usually enough.

    Requires less memory and computation.

3. Generalizes to unseen words

    New words (e.g., playfulness) can be segmented as play + ##ful + ##ness.

    Handles neologisms and inflections better.

4. More probabilistic than BPE

    Merges are chosen based on likelihood improvement, not just raw frequency.

    Produces vocabularies more aligned with the data distribution.

5. Supports multilingual settings better

    Subwords can be shared across languages.

    Example: Multilingual BERT uses WordPiece.

❌ Weaknesses

1. High computational cost for training

    Choosing merges by maximizing likelihood requires repeated re-segmentation and log-likelihood computation.

    Much slower than BPE.

2. Deterministic, no segmentation diversity

    Always uses greedy longest-match-first during inference.

    Unlike Unigram/SentencePiece, which can sample multiple segmentations (useful for data augmentation).

3. Strongly depends on preprocessing

    Lowercasing, normalization, and whitespace splitting heavily affect results.

    Problematic for languages without clear word boundaries (Chinese, Persian with half-spaces, etc.).

4. Not globally optimal

    Greedy selection of merges means the final vocabulary may not be the true optimum.

5. Longer sequences in some languages

    For highly inflected or agglutinative languages (Turkish, Finnish, Persian), words may split into many subwords.

6. Longer sequences → more expensive for Transformers (complexity o(n**2))

Subwords often lack clear semantic meaning

Tokens like ##ing, ##tion carry little standalone meaning.

Can make interpretation harder. 

### Libraries and language models that use WordPiece

- **Libraries**
  - **Hugging Face Transformers/Tokenizers**: Implements WordPiece tokenization (and others) and provides pretrained tokenizers.
  - **TensorFlow Text**: WordPiece ops for TF models (used with BERT-style models).
  - **KerasNLP**: WordPiece tokenizer layers compatible with Keras models.

- **Language models (WordPiece-based)**
  - **BERT family**: `bert-base-uncased`, `bert-base-cased`, `bert-large-*`, `bert-base-multilingual-uncased/cased` (mBERT), `BioBERT`, etc.
  - **DistilBERT**: `distilbert-base-uncased` (distilled BERT; still WordPiece).
  - **ELECTRA**: `google/electra-*` models use WordPiece.
  - **MobileBERT**: `google/mobilebert-*` uses WordPiece.
  - **LaBSE** and many multilingual encoders derived from BERT use WordPiece.

- Models that do NOT use WordPiece (for contrast)
  - **RoBERTa** and **GPT-2/3**: BPE (Byte-Pair Encoding).
  - **ALBERT** and many Google/XLNet variants may use **SentencePiece (Unigram)**.

This is why tokenization behavior (subword splits, vocabulary size) can differ across otherwise similar encoders.


In [2]:
# Comparison of tokenizers on the same input
# - WordPiece (BERT, DistilBERT, ELECTRA)
# - BPE (RoBERTa)
# - Unigram/SentencePiece (ALBERT)

from typing import List, Dict
from dataclasses import dataclass

from transformers import AutoTokenizer

@dataclass
class TokenizerSpec:
    name: str
    model_id: str
    family: str


def compare_tokenizers(text: str, specs: List[TokenizerSpec]) -> List[Dict]:
    results = []
    for spec in specs:
        tok = AutoTokenizer.from_pretrained(spec.model_id)
        # Use fast path when available
        enc = tok(text, add_special_tokens=False)
        pieces = tok.convert_ids_to_tokens(enc["input_ids"]) if hasattr(tok, "convert_ids_to_tokens") else enc["input_ids"]
        results.append({
            "name": spec.name,
            "family": spec.family,
            "model_id": spec.model_id,
            "num_tokens": len(pieces),
            "tokens": pieces,
        })
    return results


if __name__ == "__main__":
    sample_text = "Playing players play playful plays in playgrounds."

    SPECS = [
        # WordPiece
        TokenizerSpec(name="BERT (WordPiece)", model_id="bert-base-uncased", family="WordPiece"),
        TokenizerSpec(name="DistilBERT (WordPiece)", model_id="distilbert-base-uncased", family="WordPiece"),
        TokenizerSpec(name="ELECTRA (WordPiece)", model_id="google/electra-base-discriminator", family="WordPiece"),
        # BPE
        TokenizerSpec(name="RoBERTa (BPE)", model_id="roberta-base", family="BPE"),
        # SentencePiece/Unigram
        TokenizerSpec(name="ALBERT (SentencePiece)", model_id="albert-base-v2", family="SentencePiece"),
    ]

    rows = compare_tokenizers(sample_text, SPECS)

    # Pretty print comparison
    for row in rows:
        print("==>", row["name"], f"[{row['family']}]")
        print("Tokens (", row["num_tokens"], "):")
        print(row["tokens"])
        print()



/home/sherin/miniconda3/envs/exercise_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


==> BERT (WordPiece) [WordPiece]
Tokens ( 9 ):
['playing', 'players', 'play', 'playful', 'plays', 'in', 'playground', '##s', '.']

==> DistilBERT (WordPiece) [WordPiece]
Tokens ( 9 ):
['playing', 'players', 'play', 'playful', 'plays', 'in', 'playground', '##s', '.']

==> ELECTRA (WordPiece) [WordPiece]
Tokens ( 9 ):
['playing', 'players', 'play', 'playful', 'plays', 'in', 'playground', '##s', '.']

==> RoBERTa (BPE) [BPE]
Tokens ( 9 ):
['Playing', 'Ġplayers', 'Ġplay', 'Ġplayful', 'Ġplays', 'Ġin', 'Ġplayground', 's', '.']

==> ALBERT (SentencePiece) [SentencePiece]
Tokens ( 9 ):
['▁playing', '▁players', '▁play', '▁playful', '▁plays', '▁in', '▁playground', 's', '.']



### Byte-Pair Encoding (BPE)

BPE builds a fixed subword vocabulary by iteratively merging the most frequent adjacent symbol pairs in a corpus.

- Start with each word split into characters plus an end-of-word marker (e.g., `</w>`)
- Count pair frequencies across the corpus
- Merge the most frequent pair into a new symbol; repeat until reaching the target vocab size

Given a word, tokenization applies the learned merges greedily to reconstruct subword units.

Compared to WordPiece, BPE chooses merges by raw pair frequency rather than maximizing likelihood.


In [ ]:
# Classical BPE implementation (training + tokenization)
from collections import Counter, defaultdict
from typing import Dict, List, Tuple, Iterable
import re

EOW = "</w>"


def get_vocab(words: Iterable[str]) -> Counter:
    vocab = Counter()
    for w in words:
        # word as list of chars + end-of-word
        pieces = list(w) + [EOW]
        vocab[tuple(pieces)] += 1
    return vocab


def get_pair_frequencies(vocab: Counter) -> Dict[Tuple[str, str], int]:
    pair_freq: Dict[Tuple[str, str], int] = defaultdict(int)
    for word_tuple, count in vocab.items():
        for i in range(len(word_tuple) - 1):
            pair = (word_tuple[i], word_tuple[i + 1])
            pair_freq[pair] += count
    return pair_freq


def merge_pair_in_vocab(vocab: Counter, pair_to_merge: Tuple[str, str]) -> Counter:
    new_vocab: Counter = Counter()
    bigram = re.escape(" ".join(pair_to_merge))
    pattern = re.compile(r"(?<!\S)" + bigram + r"(?!\S)")
    for word_tuple, count in vocab.items():
        symbols = list(word_tuple)
        # Convert to space-separated symbols to replace bigram
        s = " ".join(symbols)
        merged_symbol = pair_to_merge[0] + pair_to_merge[1]
        s = re.sub(pattern, merged_symbol, s)
        new_symbols = tuple(s.split(" "))
        new_vocab[new_symbols] += count
    return new_vocab


def train_bpe(words: Iterable[str], num_merges: int) -> List[Tuple[str, str]]:
    vocab = get_vocab(words)
    merges: List[Tuple[str, str]] = []
    for _ in range(num_merges):
        pair_freq = get_pair_frequencies(vocab)
        if not pair_freq:
            break
        best_pair, best_count = max(pair_freq.items(), key=lambda x: x[1])
        if best_count <= 0:
            break
        merges.append(best_pair)
        vocab = merge_pair_in_vocab(vocab, best_pair)
    return merges


def apply_bpe_tokenize(word: str, merges: List[Tuple[str, str]]) -> List[str]:
    symbols = list(word) + [EOW]
    # Build a quick lookup for merges: left -> possible rights with rank
    merge_ranks: Dict[Tuple[str, str], int] = {pair: i for i, pair in enumerate(merges)}

    while True:
        # Find all adjacent pairs
        pairs = [(symbols[i], symbols[i + 1]) for i in range(len(symbols) - 1)]
        # Rank them
        candidate_pairs = [(merge_ranks.get(p, float("inf")), p) for p in pairs]
        best_rank, best_pair = min(candidate_pairs, key=lambda x: x[0])
        if best_rank == float("inf"):
            break
        # Merge first occurrence of best_pair
        new_symbols: List[str] = []
        i = 0
        while i < len(symbols):
            if i < len(symbols) - 1 and (symbols[i], symbols[i + 1]) == best_pair:
                new_symbols.append(symbols[i] + symbols[i + 1])
                i += 2
            else:
                new_symbols.append(symbols[i])
                i += 1
        symbols = new_symbols
    # Drop end-of-word marker
    if symbols and symbols[-1] == EOW:
        symbols = symbols[:-1]
    return symbols


def bpe_tokenize(text: str, merges: List[Tuple[str, str]]) -> List[str]:
    tokens: List[str] = []
    for word in re.findall(r"\S+", text):
        tokens.extend(apply_bpe_tokenize(word, merges))
    return tokens


# Minimal usage example
if __name__ == "__main__":
    corpus = [
        "Playing players play playful plays",
        "Playground played",
    ]
    # Build a small merge list for demo
    words = [w.lower() for line in corpus for w in re.findall(r"\S+", line)]
    merges = train_bpe(words, num_merges=50)
    print("#merges:", len(merges))
    print(bpe_tokenize("players", merges))



✅ Strengths (BPE)

1. **Simple and fast to train**
   - Only counts pair frequencies and merges most frequent pairs.

2. **Reduces OOVs**
   - Any word can be split into frequent subwords/characters.

3. **Compact vocabularies**
   - 30k–50k merges are usually enough for many languages.

4. **Deterministic inference**
   - Greedy application of merges yields consistent tokenization.

❌ Weaknesses (BPE)

1. **No likelihood objective**
   - Chooses merges purely by frequency, not by maximizing corpus likelihood.

2. **May overfit frequent pairs**
   - Common bigrams dominate early; rare but semantically meaningful units can be missed.

3. **Whitespace and normalization sensitive**
   - Preprocessing choices strongly affect learned merges.

4. **Subword boundary artifacts**
   - BPE encodings (like RoBERTa) use special markers (e.g., `Ġ`) to indicate word starts, which can be unintuitive.


### Libraries and models that use BPE

- **Libraries**
  - **Hugging Face Tokenizers/Transformers**: BPE training/decoding (e.g., RoBERTa, GPT-2 style)
  - **SentencePiece**: Implements BPE and Unigram (used by many Google models)

- **Language models (BPE-based)**
  - **RoBERTa** (`roberta-base`, `xlm-roberta-base` uses a BPE-like variant over bytes)
  - **GPT-2** and many GPT derivatives (OpenAI GPT-2 BPE)
  - **Llama 1/2/3** use a byte-level BPE variant
  - Many tokenizers described as "byte-level BPE" fall in this family


### SentencePiece (Unigram)

SentencePiece is a language-independent subword tokenizer that learns directly from raw text without external tokenization. Its most common mode uses a Unigram language model to select a subword vocabulary and segmentations.

Core ideas
- Operates on raw text (often bytes or Unicode codepoints), not on pre-tokenized words
- Learns a subword vocabulary and probabilities under a Unigram LM
- At inference, segments text by maximizing the likelihood under the Unigram model (can also sample for augmentation)
- Uses special markers, e.g., `▁` to indicate word boundaries in many models

Variants
- `model_type="unigram"` (default, probabilistic; supports sampling)
- `model_type="bpe"` (BPE training inside SentencePiece)
- `model_type="char"`, `word` for other baselines


In [ ]:
# Minimal SentencePiece Unigram training + tokenization demo
# Requires: pip install sentencepiece

import os
import tempfile
from typing import List

import sentencepiece as spm

CORPUS = [
    "Playing players play playful plays",
    "Playground played",
]

# Train a tiny unigram model on the toy corpus
with tempfile.TemporaryDirectory() as td:
    corpus_path = os.path.join(td, "corpus.txt")
    with open(corpus_path, "w", encoding="utf-8") as f:
        for line in CORPUS:
            f.write(line + "\n")

    model_prefix = os.path.join(td, "sp_demo")
    spm.SentencePieceTrainer.train(
        input=corpus_path,
        model_prefix=model_prefix,
        vocab_size=200,
        model_type="unigram",
        character_coverage=1.0,
        input_sentence_size=0,
        shuffle_input_sentence=False,
    )

    sp = spm.SentencePieceProcessor()
    sp.load(model_prefix + ".model")

    sample_text = "Playing players play playful plays in playgrounds."
    ids: List[int] = sp.encode(sample_text, out_type=int)
    pieces: List[str] = sp.id_to_piece(ids)

    print("SentencePiece (Unigram) tokens:")
    print(pieces)
    print("num_tokens:", len(pieces))



✅ Strengths (SentencePiece)

1. **Language independent**
   - No need for whitespace tokenization or language-specific preprocessing.

2. **Probabilistic segmentation (Unigram)**
   - Can sample multiple segmentations (useful for data augmentation).

3. **Great OOV handling**
   - Learns subwords directly over raw text; robust to scripts without spaces.

4. **Unified tooling**
   - Single trainer supports Unigram/BPE/Char, integrates cleanly in pipelines.

❌ Weaknesses (SentencePiece)

1. **Training time and memory**
   - Unigram optimization over large corpora can be heavier than BPE.

2. **Interpretability**
   - `▁` boundary markers and probabilistic outputs can be less intuitive.

3. **Pre/Post-processing differences**
   - Models trained with different normalization/coverage settings are not drop-in compatible.


### Libraries and models that use SentencePiece

- **Libraries**
  - **SentencePiece** (reference C++/Python library)
  - **Hugging Face Transformers/Tokenizers** (loads SentencePiece models; many tokenizers depend on `.model` files)

- **Language models (SentencePiece-based)**
  - **ALBERT** (`albert-base-v2`)
  - **XLNet**
  - **T5/mT5** (Unigram SentencePiece)
  - **MarianMT**
  - **PEGASUS**
  - Many multilingual encoders/MT models from Google and others
